In [ ]:
!pip install roboflow
!pip install ultralytics
import yaml
import os
from roboflow import Roboflow
from ultralytics import YOLO
from IPython.display import clear_output
from google.colab import drive

# --- ¡NUEVO PASO! Montar Google Drive ---
print("Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Drive Montado.")

# --- 1. Descargar tu NUEVO Dataset Aumentado desde Roboflow ---
print("Descargando tu nuevo dataset desde Roboflow...")
rf = Roboflow(api_key="tZR7XZhFvySrbVM3hfH4") # Tu API key
project = rf.workspace("luchini").project("datasetenproceso-irzcf")

# ----- ¡ACCIÓN REQUERIDA! -----
# (la que tiene los aumentos de Roboflow y la división 92/5/3)
version = project.version(15)
dataset = version.download("yolov8")

# --- 2. Corrección Automática del data.yaml (para evitar errores de ruta) ---
data_yaml_path = f'{dataset.location}/data.yaml'
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

data_yaml['train'] = f"{dataset.location}/train/images"
data_yaml['val'] = f"{dataset.location}/valid/images"
if os.path.exists(f"{dataset.location}/test/images"):
    data_yaml['test'] = f"{dataset.location}/test/images"

with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

clear_output()
print(f"✅ Dataset (versión {version.version}) descargado y listo.")
print(f"La ubicación del config es: {data_yaml_path}")

model = YOLO('yolov8l-seg.pt')

results = model.train(
    data=data_yaml_path,
    epochs=300,
    patience=120,           # más paciencia para mejoras tardías
    batch=16,               # 16 o 24 si entra
    imgsz=1280,
    rect=True,

    # Optimizador y LR schedule
    optimizer='SGD',
    lr0=0.0025,             # ~0.01 * (16/64)
    lrf=0.01,               # LR final = 1% del inicial
    momentum=0.937,
    weight_decay=0.0005,
    cos_lr=True,
    warmup_epochs=3,

    # Aumentos más “conservadores” para preservar color
    degrees=7.0,
    translate=0.08,
    scale=0.2,
    shear=2.0,
    flipud=0.0,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.25,
    hsv_v=0.20,
    mosaic=0.5,
    mixup=0.0,
    copy_paste=0.0,
    close_mosaic=15,        # desactiva mosaic al final

    # Regularización ligera
    label_smoothing=0.05,   # ayuda si hay ruído entre maduro/sobremaduro

    project='/content/drive/MyDrive/YOLOv8_Training',
    name='Coffee_Seg_L-1280_SGDcos_AugsConservadores_v1',
    exist_ok=True,
)


✅ Dataset (versión 15) descargado y listo.
La ubicación del config es: /content/DatasetEnProceso-15/data.yaml
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=15, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/DatasetEnProceso-15/data.yaml, degrees=7.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.25, hsv_v=0.2, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=t